In [ ]:
# Installation des packages nécessaires, en utilisant packagemanager.posit pour accélérer
options(repos=list(CRAN='https://packagemanager.posit.co/cran/__linux__/jammy/latest/'))
install.packages('rlibkriging')
install.packages('DiceView')

In [ ]:
# Latest DiceView (waiting for CRAN sync)
remotes::install_github('IRSN/DiceView')

In [ ]:
#knitr::opts_chunk$set(echo = TRUE)

# chargement des librairies
library(rlibkriging)
library(DiceView)

# setup default plots aesthetics
par(oma = c(0,0,0,0))
par(mar = c(3, 3, 1, 1))
par(cex.lab=0.5, cex.axis=0.5, cex.main=0.5)
par(bg = "white")

options(digits=5)

# Objectif

On cherche à minimiser la fonction
$$ f: x \rightarrow -(1.4-3 x) sin(18 x)$$

Soit, identifier

$$x_{min} = argmin_{x \in D_X}(f(.)) $$

In [ ]:
f <- function(x)
    -(1.4 - 3*x) * sin(18*x)

plot(f)

# On cherche à identifier xmin:
xmin = optims(c(0.95,1.), f, method="L-BFGS-B", lower=0, upper=1)$par
abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)

# Métamodèle GP

Sur un plan d'expérience inital de petite taille $(X,Y)$, on construit un processus gaussien :
$$\xi_{X,Y}: x \rightarrow \xi(x) \sim \mathcal N(m_{X,Y}(x),k_{X,Y}(x)).$$

Ce méta-modèle est insuffisant en l'état pour atteindre l'objectif :

In [ ]:
# plan d'exp initial
X0 = matrix(c(.01, 0.31, 0.82, 0.46, 0.61),ncol=1)
Y0 = f(X0)



GP = function(X,Y)
  Kriging(matrix(Y), matrix(X), kernel="gauss", optim="none", parameters=list(theta=c(.15),sigma2=1))

sectionview(GP(X0,Y0), Xlim=c(0,1),ylim=c(-2,1.5))
sectionview(f, dim=1, col='black', Xlim=c(0,1),ylim=c(-2,1.5),add = TRUE)
points(X0, Y0)
# Objectif
abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)

# Amélioration espérée du minimum courant du métamodèle
Soit $X,Y$ notre plan d'expériences courant (pour le moment initial).

Choix du critère "local" d'intérêt : Espérance de dépassement du minimum courant.
$$EI(x|X,Y):=E_{\xi_{X,Y}(x)}[min(Y) - \xi_{X,Y}(x)]^{+} $$

On calcule le critère "global" $J$ en utilisant les prédictions du GP courant $\xi$ et en intégrant numériquement le critère sur $D_X$ :

$$J_{X,Y} := \int_{x \in D_X}EI(x|X,Y)dx$$

In [ ]:
x = seq(0, 1,,101)

X = X0
Y = Y0

# predictions du métamodèle
y = GP(X,Y)$predict(x)
minY=min(Y)

# amélioration espérée
g <- (minY - y$mean) / y$stdev
g[y$stdev==0] = 0
EI = y$stdev * (g * pnorm(g) + dnorm(g))

par(mfrow=c(2,1))

sectionview(GP(X,Y), Xlim=c(0,1),ylim=c(-2,1.5), conf_lev=.9)

abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)

abline(h=min(Y), col='red',lty=2)

plot(x,EI, type='l',col='violet', main="Amélioration espérée du min courant")
polygon(c(0,x,1), c(0,EI,0), col=rgb(1,0.5,1,.2), border=NA)
abline(v=xmin, col='red',lty=3)

# J: intégrale sur X de l'amélioration espérée
text(x=.5,y=.05,labels=paste0("J = ",sum(EI)/length(EI)))



Une alternative consiste en un calcul empirique utilisant les simulations du GP (et non pas les prédictions)

In [ ]:
# simulations du métamodèle
S = GP(X,Y)$simulate(nsim=100,seed=123, x)
#print(dim(S))

# amélioration espérée
ppos = function(x) {x[x<0]=0; x}
#print(GP(X,Y)$y())
minY=min(GP(X,Y)$y())
EI_S = rowMeans(ppos(minY - S))

par(mfrow=c(2,1))

sectionview(GP(X,Y), Xlim=c(0,1),ylim=c(-2,1.5), conf_lev=.9, conf_fading = 0.1)
for (i in 1:ncol(S))
    lines(x,S[,i], col=rgb(0,0,1,.21),lty=1)
# Objectif
abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)
abline(h=min(Y), col='red',lty=2)

plot(x,EI_S, type='l',col='violet', main="Amélioration espérée du min courant",lwd=3)
polygon(c(0,x,1), c(0,EI_S,0), col=rgb(1,0.5,1,.2), border=NA)
lines(x,EI, type='l',lty=3,col='violet', main="Amélioration espérée du min courant",lwd=3)
# Objectif
abline(v=xmin, col='red',lty=3)

# J: intégrale sur X de l'amélioration espérée
text(x=.5,y=.05,labels=paste0("Ĵ = ",sum(EI_S)/length(EI_S)))

#(optim) Recherche d'un nouveau point de design...

Considérons à présent notre critère d'intérêt : "espérance de dépassement du minimum courant intégré", comme une fonction du plan d'expériences
$$J(X,Y) = \int_{x \in D_X} EI(x|X,Y)dx$$



In [ ]:
J = function(X,Y) {
    y = GP(X,Y)$predict(x)
    g <- (min(Y) - y$mean) / y$stdev
    g[y$stdev==0] = 0
    EI = y$stdev * (g * pnorm(g) + dnorm(g))
    return(mean( EI ))
}
J(X,Y)

---
#Démarche "SUR"
Nous cherchons à placer un point de design qui ajouté au plan d'expériences,
$$({X \cup x_{next}},{Y \cup y_{next}}),$$
va minimiser notre critère global :
$$J({X \cup x_{next}},{Y \cup y_{next}})$$
En supposant que les valeurs $y_{next}$ sont des réalisations de $\xi_{X,Y}(x_{next})$, le GP courant, nous introduisons l'amélioration espérée correspondante :

$$J_{next}(x_{next}) = E_{\xi(x_{next})} \big[J({X \cup x_{next}},{Y \cup \xi(x_{next})}\big].$$

---

Soit,

\begin{eqnarray}
J_{next}(x_{next}) &=& \int_{y_{next} \sim \xi_{X,Y}(x_{next})} \int_{x \in D_X} EI(x|{X \cup x_{next}},{Y \cup y_{next}})dx&\\
&=& \int_{y_{next} \sim \xi_{X,Y}(x_{next})} \int_{x \in D_X} E_{\xi_{next}(x)}[min(Y \cup y_{next}) - \xi_{next}(x)]^{+}dx&
\end{eqnarray}

où $\xi_{next}$ est le PG conditionné en $Y \cup y_{next}$.

Evaluation de $J_{next}(x_{next})$ et plot de $EI(x|{X \cup x_{next}},{Y \cup y_{next}})$ en un point $x_{next}$ donné et pour quelques réalisations de $y_{next}$ :

In [ ]:
#split.screen(figs = c(2,1))

#screen(1, new=TRUE)
plot(f, xlim=c(0,1), ylim=c(-2,1.5))
sectionview(GP(X,Y), conf_lev=.9, conf_fading = 0.5, Xlim=c(0,1), ylim=c(-2,1.5), add=TRUE)
# Objectif
abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)

#screen(2, new=TRUE)
.x=seq(0,1,,1001)
y.x = GP(X,Y)$predict(.x)
g <- (min(Y) - y.x$mean) / y.x$stdev
g[y.x$stdev==0] = 0
EI.x = y.x$stdev * (g * pnorm(g) + dnorm(g)) *10
#lines(.x,EI.x, type='l',col='blue')
polygon(c(0,.x,1), c(0,EI.x,0)-2, col=rgb(0,0,1,.2), border=NA)
text(x=.5,y=-1.5,col="blue",labels=paste0("J = ",J(X,Y)))

# choix arbitraire du point de design suivant
xnext = x[16] # =0.15
#screen(1, new=FALSE)
abline(v=xnext,col='red')

# calcul du SUR avec ce point
Ynext = GP(X,Y)$predict(xnext)

Jnext = array(NA,5)
for (i in 1:length(Jnext)) { # on échantillone ynext selon la loi de prédiction en xnext (normale)
    q = i/(length(Jnext)+1)
    ynext_i = qnorm(q, Ynext$mean, Ynext$stdev)
    jnext_i = J(c(X,xnext),c(Y,ynext_i))
    Jnext[i] = jnext_i

    GPnext_i = GP(c(X,xnext),c(Y,ynext_i))
    #screen(1, new=FALSE)
    sectionview(GPnext_i, col="violet", conf_lev=.9, conf_fading = 0.5, Xlim=c(0,1), add=TRUE)

    #screen(2, new=FALSE)
    yn.x = GPnext_i$predict(.x)
    gn <- (min(c(Y,ynext_i)) - yn.x$mean) / yn.x$stdev
    gn[yn.x$stdev==0] = 0
    EI.x = yn.x$stdev * (gn * pnorm(gn) + dnorm(gn)) *10
    #lines(.x,p.x*(1-p.x),col='violet')
    polygon(c(0,.x,1), c(0,EI.x,0)-2, col=rgb(1,0.5,1,.2), border="violet")
}
text(x=xnext,y=-1.2,col="violet",labels=paste0("Jnext(",xnext,") = \n",mean(Jnext)))

# Calcul de Jnext

In [ ]:
# calcul "exact" du SUR avec ce point
Ynext = GP(X,Y)$predict(xnext)
Jnext = integrate(Vectorize(function(q) # on échantillonne selon la loi (normale) de Kn en xnext
    J(c(X,xnext),c(Y, qnorm(q, Ynext$mean, Ynext$stdev)))),0 +1e-3, 1 -1e-3)$value
print(paste0("Jnext(",xnext,")  = ",Jnext))

# ... ou version approx. par mesh régulier des quantiles :
Jnext = function(xnext, X, Y) {
    if (any(abs(xnext-X)<1e-5)) { return(J(X,Y)) }
    Ynext = GP(X,Y)$predict(xnext)
    Jnext = array(NA,100)
    for (i in 1:length(Jnext)) {
        q = i/(length(Jnext)+1)
        ynext_i = qnorm(q, Ynext$mean, Ynext$stdev)
        Jnext[i] = J(c(X,xnext),c(Y,ynext_i))
    }
    return(mean(Jnext))
}
print(paste0("Jnext(",xnext,") ~= ",Jnext(xnext,X,Y)))

# Critère local vs. global

On peut comparer les deux approches "locale" $EI$ et "globale" $J_{next}$ de choix du prochain point de design :

In [ ]:
par(mfrow=c(3,1))

sectionview(GP(X,Y), Xlim=c(0,1),ylim=c(-2,1.5), conf_lev=.9)
# Objectif
abline(v=xmin, col='red',lty=3)
points(xmin,f(xmin), col='red',pch=20)

# predictions du métamodèle
y = GP(X,Y)$predict(x)

# amélioration espérée
g <- (min(Y) - y$mean) / y$stdev
g[y$stdev==0] = 0
EI = y$stdev * (g * pnorm(g) + dnorm(g))

abline(h=min(Y), col='red',lty=2)

plot(x,EI, type='l',col='violet', main="Amélioration espérée du minimum courant")
polygon(c(0,x,1), c(0,EI,0), col=rgb(1,0.5,1,.2), border=NA)
# Objectif
abline(v=xmin, col='red',lty=3)

# meilleurs points (selon EI):
P_opts = optims(seq(0,1,,15), function(x) {
  y = GP(X,Y)$predict(x);
  g = (min(Y) - y$mean) / y$stdev;
  g[y$stdev==0] = 0;
  ei = y$stdev * (g * pnorm(g) + dnorm(g));
  -ei }, method="L-BFGS-B", lower=0, upper=1)

P_opts = P_opts$pars[P_opts$values < -0.00001]

abline(v=P_opts,col='violet',lty=3)

DJ_x = J(X,Y) - Vectorize(function(.x) Jnext(.x,X,Y))(x)
plot(x,DJ_x, type='l',col='purple', main="SUR EI")
polygon(c(0,x,1), c(0,DJ_x,0), col=rgb(0.5,0,1,.2), border=NA)
abline(v=P_opts,col='violet',lty=3)
# Objectif
abline(v=xmin, col='red',lty=3)

# Utilisation des simulations pour évaluer le critère Jnext

Fonction générique de calcul de $\hat J(X,Y)$ à partir de simulations du processus:


In [ ]:
J_hat = function(X,Y, nsim=100) {
    #y = GP(X,Y)$predict(x)
    S = GP(X,Y)$simulate(nsim=nsim,seed=123, x)
    EI = rowMeans(ppos(min(Y) - S))
    j = mean( EI )
    attributes(j) <- list("sd" = sd(EI)/sqrt(nsim))
    return(j)
}
t0 = Sys.time()
print(paste0("Calcul avec les prédictions: J = ",J(X0,Y0)))
print(Sys.time()-t0)

t0 = Sys.time()
Jhat = J_hat(X0,Y0, nsim=10000)
print(paste0("Calcul avec les simulations: Ĵ = ",Jhat, " +/- ",attributes(Jhat)$sd))
print(Sys.time()-t0)

On constate que l'utilisation des simulations (pour une précision élevée) est plus __lente__ que l'utilisation des prédictions.

Utilisons l'approche par simulation pour la fonction générique de calcul de $\hat J_{next}$ :

In [ ]:
Jnext_hat = function(xnext, X,Y, nsim=100) {
    if (any(abs(xnext-X)<1e-5)) { return(J_hat(X,Y)) }
    Ynext = GP(X,Y)$predict(xnext)
    Jnext = array(NA,100)
    for (i in 1:length(Jnext)) { # on échantillone ynext selon la loi de prédiction en xnext (normale)
        q = i/(length(Jnext)+1)
        ynext_i = qnorm(q, Ynext$mean, Ynext$stdev)
        Jnext[i] = J_hat(c(X,xnext),c(Y,ynext_i), nsim=nsim)
    }
    return(mean(Jnext))
}
t0 = Sys.time()
print(paste0("Calcul avec les prédictions: Jnext(",xnext,") = ",Jnext(xnext,X0,Y0)))
print(Sys.time()-t0)

t0 = Sys.time()
print(paste0("Calcul avec les simulations: Ĵnext(",xnext,") = ",Jnext_hat(xnext,X0,Y0, nsim=100)))
print(Sys.time()-t0)

On optimise le calcul de $\hat J_{next}$ en utilisant les updates de simulations du GP :

In [ ]:
Jnext_hat_fast = function(xnext, X,Y, nsim=100) {
    if (any(abs(xnext-X)<1e-5)) { return(J_hat(X,Y)) }
    Ynext = GP(X,Y)$predict(xnext)
    gp = GP(X,Y)
    S = gp$simulate(nsim=nsim,seed=123, c(xnext,x), will_update=TRUE) # needed for later 'update_simulate'
    Jnext = array(NA,100)
    for (i in 1:length(Jnext)) { # on échantillone ynext selon la loi de prédiction en xnext (normale)
        q = i/(length(Jnext)+1)
        ynext_i = qnorm(q, Ynext$mean, Ynext$stdev)
################################################################################
        Snext_i = gp$update_simulate(ynext_i, xnext)
################################################################################
        EI = rowMeans(ppos(min(c(Y,ynext_i)) - Snext_i))
        Jnext[i] = mean( EI )
    }
    return(mean(Jnext))
}
t0 = Sys.time()
print(paste0("Calcul avec les prédictions: Jnext(",xnext,") = ",Jnext(xnext,X0,Y0)))
print(Sys.time()-t0)

t0 = Sys.time()
print(paste0("Calcul avec les simulations: Ĵnext(",xnext,") = ",Jnext_hat(xnext,X0,Y0, nsim=100)))
print(Sys.time()-t0)

t0 = Sys.time()
print(paste0("Calcul avec les simulations updatées: Ĵnextup(",xnext,") = ",Jnext_hat_fast(xnext,X0,Y0, nsim=100)))
print(Sys.time()-t0)

On peut finalement comparer ces différentes versions du critère $J_{next}$ :

In [ ]:
par(mfrow=c(3,1))

DJ_x = J(X,Y) - Vectorize(function(.x)Jnext(.x,X,Y))(x)
plot(x,DJ_x, type='l',col='purple', main="SUR EI (predict)")
polygon(c(0,x,1), c(0,DJ_x,0), col=rgb(0.5,0,1,.2), border=NA)
# Objectif
abline(v=xmin, col='red',lty=3)

DJ_hat_x = J_hat(X,Y) - Vectorize(function(.x)Jnext_hat(.x,X,Y,nsim=100))(x)
plot(x,DJ_hat_x, type='l',col='purple', main="SUR EI (simulate)")
polygon(c(0,x,1), c(0,DJ_hat_x,0), col=rgb(0.5,0,1,.2), border=NA)
# Objectif
abline(v=xmin, col='red',lty=3)

DJ_hat_fast_x = J_hat(X,Y) - Vectorize(function(.x)Jnext_hat_fast(.x,X,Y,nsim=100))(x)
plot(x,DJ_hat_fast_x, type='l',col='purple', main="SUR EI (update simulate)")
polygon(c(0,x,1), c(0,DJ_hat_fast_x,0), col=rgb(0.5,0,1,.2), border=NA)
# Objectif
abline(v=xmin, col='red',lty=3)